Final Project:

Analysis of Bitcoin Over Time

Introduction

In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Website I got this data from:

https://www.kaggle.com/datasets/zillow/rent-index?select=pricepersqft.csv

Data Collection

Importing rent prices data

For this project, the data comes from two csv files that were posted to Kaggle and another csv file with data from the US Census. 

The two files we import bellow contain data on price per square foot and total rent price of rental properties in various cities throughout the US. We can read the file by using pandas read_csv function which will read our file and import it straight into a dataframe. Later we will combine all three.

In [133]:
price = pd.read_csv('price.csv')
price

,City Code,City,Metro,County,State,Population Rank,November 2010,December 2010,January 2011,February 2011,...,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017
0,6181,New York,New York,Queens,NY,1,NaN,NaN,NaN,NaN,...,2334,2339,2345,2344,2336,2324,2318,2321,2321,2322
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2184.0,2184.0,2183.0,2188.0,...,2637,2662,2687,2704,2716,2723,2731,2740,2748,2753
2,17426,Chicago,Chicago,Cook,IL,3,1563.0,1555.0,1547.0,1537.0,...,1684,1686,1687,1685,1681,1675,1668,1656,1644,1632
3,39051,Houston,Houston,Harris,TX,4,1198.0,1199.0,1199.0,1200.0,...,1444,1446,1446,1443,1440,1438,1437,1437,1435,1430
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,1092.0,1099.0,1094.0,1087.0,...,1206,1211,1218,1222,1223,1220,1216,1211,1209,1212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13126,397405,Highland Township,Gettysburg,Adams,PA,13127,1280.0,1280.0,1284.0,1284.0,...,1442,1440,1419,1409,1398,1395,1384,1368,1342,1321
13127,398292,Town of Wrightstown,Green Bay,Brown,WI,13128,639.0,650.0,668.0,694.0,...,1114,1133,1136,1124,1090,1047,1012,1004,1018,1047
13128,398343,Urbana,Corning,Steuben,NY,13129,1433.0,1431.0,1437.0,1458.0,...,1595,1611,1612,1607,1609,1619,1627,1625,1625,1624
13129,398839,Angels,NaN,Calaveras,CA,13130,1516.0,1529.0,1529.0,1515.0,...,1514,1528,1534,1525,1526,1529,1541,1553,1558,1563


In [134]:
price_per_sq_ft = pd.read_csv('pricepersqft.csv')
price_per_sq_ft

,City Code,City,Metro,County,State,Population Rank,November 2010,December 2010,January 2011,February 2011,...,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017
0,6181,New York,New York,Queens,NY,1,NaN,NaN,NaN,NaN,...,1.806,1.810,1.816,1.824,1.828,1.836,1.844,1.858,1.866,1.872
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,1.578,1.578,1.580,1.582,...,1.990,2.004,2.018,2.026,2.032,2.038,2.042,2.048,2.056,2.064
2,17426,Chicago,Chicago,Cook,IL,3,1.244,1.248,1.254,1.254,...,1.354,1.362,1.370,1.374,1.378,1.380,1.380,1.380,1.376,1.374
3,39051,Houston,Houston,Harris,TX,4,0.788,0.784,0.784,0.786,...,0.984,0.984,0.982,0.980,0.976,0.974,0.974,0.976,0.976,0.974
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,0.854,0.858,0.858,0.858,...,0.948,0.956,0.962,0.964,0.964,0.966,0.968,0.972,0.974,0.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11914,182023,Henrico,Richmond,Henrico,VA,11915,0.740,0.736,0.734,0.734,...,0.854,0.856,0.856,0.856,0.856,0.858,0.858,0.860,0.862,0.860
11915,397405,Highland Township,Gettysburg,Adams,PA,11916,0.780,0.776,0.778,0.778,...,0.832,0.836,0.826,0.822,0.812,0.810,0.808,0.804,0.788,0.774
11916,398343,Urbana,Corning,Steuben,NY,11917,0.954,0.956,0.970,0.984,...,1.030,1.032,1.038,1.042,1.046,1.048,1.048,1.048,1.042,1.034
11917,398839,Angels,NaN,Calaveras,CA,11918,0.976,0.984,0.988,0.980,...,0.974,0.980,0.984,0.980,0.980,0.982,0.992,1.004,1.014,1.018


Printing out our tables is veery important so that we can visualize aspects of our data. Throughout this tutorial, any time we change our tables we will display it to make sure that everything is in order.

In the above table we can already see some important information. 

We can see that our range of dates is 2011-2016, as well as that there are some NaN values. We will be adressing this information soon.

Importing median income data

We will also be importing data from a seperate csv file that contains the median house hold income each year. When I downloaded the file, I specifically chose to only download data from 2011-2016 because that is the range of dates in our other table. 

Note: We are not including the one month in 2017 because it is only one month out of the year. This will get dropped later when we merge the tables.

In [135]:
med_inc = pd.read_csv('MEHOINUSA672N.csv')
med_inc

,DATE,MEHOINUSA672N
0,2011-01-01,63350
1,2012-01-01,63350
2,2013-01-01,65740
3,2014-01-01,64900
4,2015-01-01,68410
5,2016-01-01,70840


Data Processing

At this point in the Data Science Pipeline we have our data imported into dataframes, ready to be accessed. Now we have to process our data which requires manipulating our tables to display information in an effective way. We can use Pandas and Numpy to do this. These are two very powerfull libraries that provide functionality to our data that you will see throughout this tutorial. 

We have already imported our data into a pandas dataframe which will allow us to mold it to fit our needs.

Cleaning The Data

Right now, our data has distinct columns for each month throughout our date range. This format can be dificult to work with so we are going to create a new column called "Date" which will have a distinct month and year for each observation (Row).

This will increase the number of rows we have in our dataframe by a factor of 60 (12 months over 5 years) but we will also loose those 60 seperate date columns in exchange for just one per observation.

This will be very usefull later on for grouping and aggregating data based on dates.

In [136]:
ppsf = pd.melt(price_per_sq_ft, id_vars=['City Code', 'City', 'Metro', 'County', 'State', 'Population Rank'], var_name='Date', value_name='Price Per Sq. Ft.')
price = pd.melt(price, id_vars=['City Code', 'City', 'Metro', 'County', 'State', 'Population Rank'], var_name='Date', value_name='Rent Price')
print(ppsf)
print(price)

        City Code               City         Metro        County State  \
0            6181           New York      New York        Queens    NY   
1           12447        Los Angeles   Los Angeles   Los Angeles    CA   
2           17426            Chicago       Chicago          Cook    IL   
3           39051            Houston       Houston        Harris    TX   
4           13271       Philadelphia  Philadelphia  Philadelphia    PA   
...           ...                ...           ...           ...   ...   
893920     182023            Henrico      Richmond       Henrico    VA   
893921     397405  Highland Township    Gettysburg         Adams    PA   
893922     398343             Urbana       Corning       Steuben    NY   
893923     398839             Angels           NaN     Calaveras    CA   
893924     737788    Lebanon Borough      New York     Hunterdon    NJ   

        Population Rank           Date  Price Per Sq. Ft.  
0                     1  November 2010             

Here, we are going to merge the Price Per Square Inch table and the Total Price dataframes. We want to merge on every column except the two that are different (Price per sq in and price). This way, we will ensure that we match the correct price with its price per sq in and vise versa.


We will also add a year column to be used when we merge the dataframes. When we merge them, we will do it on the year column because we only have the average income for each year and in our rent data we have multiple observations per year. This will ensure that if two observations have the same year, they also have the same median income.

In [137]:
data = pd.merge(price, ppsf, on=['City Code', 'City', 'Metro', 'County', 'State', 'Population Rank', 'Date'])
data['Date'] = pd.to_datetime(data['Date'], format='%B %Y')
data['year'] = data['Date'].dt.year
data

,City Code,City,Metro,County,State,Population Rank,Date,Rent Price,Price Per Sq. Ft.,year
0,6181,New York,New York,Queens,NY,1,2010-11-01,NaN,NaN,2010
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2010-11-01,2184.0,1.578,2010
2,17426,Chicago,Chicago,Cook,IL,3,2010-11-01,1563.0,1.244,2010
3,39051,Houston,Houston,Harris,TX,4,2010-11-01,1198.0,0.788,2010
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,2010-11-01,1092.0,0.854,2010
...,...,...,...,...,...,...,...,...,...,...
20770,17845,Eagan,Minneapolis-St Paul,Dakota,MN,644,2017-01-01,1728.0,1.538,2017
20771,17718,Daytona Beach,Daytona Beach,Volusia,FL,645,2017-01-01,1125.0,0.878,2017
20772,26483,Pico Rivera,Los Angeles,Los Angeles,CA,646,2017-01-01,2220.0,1.784,2017
20773,54043,Racine,Racine,Racine,WI,647,2017-01-01,1013.0,0.898,2017


By default, the merge function uses an inner join which only returns rows that are the same in both dataframes. This is good for us because we want our observations to include price and price per sq ft.

Now that we have cleaned up our rent data, we are going to look at the Medium Income data.

The "DATE" columns is represented by a string but we want to change it to datetime datatype as this will allow for more functionality. One reason is for merging this table with our other data and to do that we need identical datatypes.

We also have to create a year column for when we merge the two dataframes.

In [138]:
med_inc['Date'] = pd.to_datetime(med_inc['DATE'])
med_inc.drop('DATE', axis=1, inplace=True)
med_inc['year'] = med_inc['Date'].dt.year
med_inc.drop('Date', axis=1, inplace=True)
med_inc

,MEHOINUSA672N,year
0,63350,2011
1,63350,2012
2,65740,2013
3,64900,2014
4,68410,2015
5,70840,2016


Adding this data to the main dataframe

Bellow, we merge the two dataframes with Pandas merge function. We acan take advantage of the year columns we made in the two dataframes. However, once we are done with the year column we can drop it as we wont use it again.

In [139]:
data = pd.merge(data, med_inc, on='year')
data = data.rename(columns={'MEHOINUSA672N' : 'Average Income'})
data.drop('year', axis=1, inplace=True)
data

,City Code,City,Metro,County,State,Population Rank,Date,Rent Price,Price Per Sq. Ft.,Average Income
0,6181,New York,New York,Queens,NY,1,2011-01-01,NaN,NaN,63350
1,12447,Los Angeles,Los Angeles,Los Angeles,CA,2,2011-01-01,2183.0,1.580,63350
2,17426,Chicago,Chicago,Cook,IL,3,2011-01-01,1547.0,1.254,63350
3,39051,Houston,Houston,Harris,TX,4,2011-01-01,1199.0,0.784,63350
4,13271,Philadelphia,Philadelphia,Philadelphia,PA,5,2011-01-01,1094.0,0.858,63350
...,...,...,...,...,...,...,...,...,...,...
19939,17845,Eagan,Minneapolis-St Paul,Dakota,MN,644,2016-12-01,1723.0,1.532,70840
19940,17718,Daytona Beach,Daytona Beach,Volusia,FL,645,2016-12-01,1123.0,0.876,70840
19941,26483,Pico Rivera,Los Angeles,Los Angeles,CA,646,2016-12-01,2203.0,1.770,70840
19942,54043,Racine,Racine,Racine,WI,647,2016-12-01,1021.0,0.896,70840


We can see above that we lost a few rows. This is because when we merged that median income (ranging from 2011-2016) it dropped the rows from 2017.

NaN Values

Now we will adress the rows with NaN values.

There are a couple ways we can deal with rows that have missing data. We can either delete those rows, or insert values that reflect similar observations or averages. 

First, we should look at the rows with NaN values to decide what we should do.

In [140]:
rows_with_nan = data[data.isna().any(axis=1)]
rows_with_nan

,City Code,City,Metro,County,State,Population Rank,Date,Rent Price,Price Per Sq. Ft.,Average Income
0,6181,New York,New York,Queens,NY,1,2011-01-01,NaN,NaN,63350
27,5976,Milwaukee,Milwaukee,Milwaukee,WI,28,2011-01-01,NaN,NaN,63350
32,23429,Albuquerque,Albuquerque,Bernalillo,NM,33,2011-01-01,NaN,NaN,63350
47,44055,Baton Rouge,Baton Rouge,East Baton Rouge,LA,48,2011-01-01,NaN,NaN,63350
48,19594,New Orleans,New Orleans,Orleans,LA,49,2011-01-01,NaN,NaN,63350
...,...,...,...,...,...,...,...,...,...,...
19263,11896,High Point,NaN,Guilford,NC,522,2016-10-01,875.0,0.684,70840
19477,11722,Greensboro,NaN,Guilford,NC,88,2016-11-01,979.0,0.724,70840
19540,11896,High Point,NaN,Guilford,NC,522,2016-11-01,874.0,0.684,70840
19754,11722,Greensboro,NaN,Guilford,NC,88,2016-12-01,984.0,0.724,70840


We can see above that there are multiple cases on missing values. One that we can see is missing price and price per sq in values. These are vital to our observations so we can drop these columns 

In [141]:
data = data.dropna(subset=['Rent Price', 'Price Per Sq. Ft.'], axis=0)
rows_with_nan = data[data.isna().any(axis=1)]
rows_with_nan

,City Code,City,Metro,County,State,Population Rank,Date,Rent Price,Price Per Sq. Ft.,Average Income
87,11722,Greensboro,NaN,Guilford,NC,88,2011-01-01,917.0,0.660,63350
150,11896,High Point,NaN,Guilford,NC,522,2011-01-01,838.0,0.642,63350
364,11722,Greensboro,NaN,Guilford,NC,88,2011-02-01,927.0,0.670,63350
427,11896,High Point,NaN,Guilford,NC,522,2011-02-01,851.0,0.648,63350
641,11722,Greensboro,NaN,Guilford,NC,88,2011-03-01,945.0,0.680,63350
...,...,...,...,...,...,...,...,...,...,...
19263,11896,High Point,NaN,Guilford,NC,522,2016-10-01,875.0,0.684,70840
19477,11722,Greensboro,NaN,Guilford,NC,88,2016-11-01,979.0,0.724,70840
19540,11896,High Point,NaN,Guilford,NC,522,2016-11-01,874.0,0.684,70840
19754,11722,Greensboro,NaN,Guilford,NC,88,2016-12-01,984.0,0.724,70840


So now, we can see there are also rows with NaN for the "Metro" column. Lets find out why these values are NaN.

There are a few possibilities: 

Based on the data printed above, it looks like it might be one specific county that is missing values for Metro.
Another observation is that it could be related to the state.

To verify this, we can check the unique values of these columns in the rows that have NaN values. If there is only one unique county or state then we know that it is not at random.

In [142]:
print(rows_with_nan['County'].unique())
print(rows_with_nan['State'].unique())

['Guilford']
['NC']


Well, we can see that the NaN values are related to a specific county. It seems that The only NaN values left are in the Metro column of only observations from Guilford county North Carolina.

Let's now check to see if after we drop the rows with NaN values, there are any observations left from Guilford county

In [144]:
data = data.dropna(subset='Metro', axis=0)
len(data[data['County'] == 'Guilford'])

0

Since all observations from Guilford county are NaN, it is ok for us to drop the observations because we don't really need those observations from that one specific county.

In [145]:
len(data[data.isna().any(axis=1)])

0

We can see above that there are no more rows with any NaN values, now we can move on to visualizing the data.

Data Visualization